In [1]:
%%capture
import os
import pandas as pd
import numpy as np
import mne
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from scipy.signal import welch

In [2]:
%%capture
# Define a function to extract labels from filenames
def extract_label_from_filename(filename):
    # Assuming filenames are in the format: subject_task.bdf
    task = filename.split('_')[1].split('.')[0]
    return task


In [3]:
%%capture
#Convert to DataFrame
def convert_bdf_to_dataframe(bdf_filename):
    raw_data = mne.io.read_raw_bdf(bdf_filename, preload=True)
    eeg_data1 = raw_data.get_data()
    channel_names = raw_data.ch_names
    time_index = raw_data.times
    eeg_data = pd.DataFrame(data=eeg_data1.T, columns=channel_names, index=time_index)
    col_names = ['O1', 'O2', 'F3', 'F4', 'C3' ,'C4' , 'Fp1', 'Fp2']
    eeg_data = eeg_data[col_names]
    segment_size = 1024
    num_segments = len(eeg_data) // segment_size
    reduced_df = reduced_df = pd.DataFrame(columns=eeg_data.columns)
    for i in range(num_segments):
        start_idx = i * segment_size
        end_idx = (i + 1) * segment_size
        segment_data = eeg_data.iloc[start_idx:end_idx]    
        mean_values = segment_data.mean()
        sampling_frequency = 1024  
        psd_data = {}  # Dictionary to store PSD results for each electrode
        reduced_df = reduced_df.append(mean_values, ignore_index=True)

    return reduced_df, raw_data

In [4]:
%%capture
# Load and preprocess the data
data_dir = '../Dataset/'  # Replace with the path to your .bdf data directory

# Initialize empty lists to store data and labels
data = []
labels = []
raw_eegdata = []

# Loop through all files in the directory
for filename in os.listdir(data_dir):
    if filename.endswith('.bdf'):
        # Convert the .bdf file to a DataFrame
        eeg_data, eeg_raw = convert_bdf_to_dataframe(os.path.join(data_dir, filename))
        
        # Extract labels from filenames
        label = extract_label_from_filename(filename)
        
        # Append data and labels
        data.append(eeg_data)
        raw_eegdata.append(eeg_raw)
        labels.extend([label] * len(eeg_data))
        channel_name_1 = ['O1', 'O2']
        channel_name_2 = ['F3', 'F4','C3' ,'C4' , 'Fp1', 'Fp2']  # Add more channel names as needed

        # Create dictionaries to store the filtered data
        eeg_dataframe = pd.DataFrame()


        # Loop through each channel and filter data
        for channel_name in channel_name_2:
            eeg_channel = eeg_raw.copy().pick_channels([channel_name])
    
        # Filter for Alpha (8-13 Hz)
            alpha_filtered = eeg_channel.filter(l_freq=8, h_freq=13)
            alpha_decomp = alpha_filtered.get_data()[0]
            eeg_dataframe[f'{channel_name} alpha'] = alpha_decomp
    
        # Filter for Beta (13-30 Hz)
            beta_filtered = eeg_channel.filter(l_freq=13, h_freq=30)
            beta_decomp = beta_filtered.get_data()[0]
            eeg_dataframe[f'{channel_name} beta'] = beta_decomp
    
        for channel_name in channel_name_1:
            eeg_channel = eeg_raw.copy().pick_channels([channel_name])
    
        # Filter for Gamma (30-40 Hz)
            gamma_filtered = eeg_channel.filter(l_freq=30, h_freq=40)
            gamma_decomp = gamma_filtered.get_data()[0]
            eeg_dataframe[f'{channel_name} gamma'] = gamma_decomp


        segment_size = 1024
        num_segments = len(eeg_dataframe) // segment_size
        reduced_decomp_df = pd.DataFrame(columns=eeg_dataframe.columns)
        for i in range(num_segments):
            start_idx = i * segment_size
            end_idx = (i + 1) * segment_size
            segment_data = eeg_dataframe.iloc[start_idx:end_idx]    
            mean_values = segment_data.mean()
            sampling_frequency = 1024  
            reduced_decomp_df = reduced_decomp_df.append(mean_values, ignore_index=True)
        def extract_mean_and_psd_mean(reduced_decomp_df):
            new_df = pd.DataFrame(columns=['channel', 'mean', 'psd_mean'])
            req_channels = reduced_decomp_df.columns
#     df = pd.read_csv(csv_file)
            for channel in req_channels:
                if channel not in req_channels:
                    continue
            mean_val = reduced_decomp_df[channel].mean()
            _, psd = welch(reduced_decomp_df[channel], fs=256)
            new_rec = {
                'channel': channel,
                'mean': mean_val,
                'psd_mean': psd.mean()
            }
            new_df = new_df.append(new_rec, ignore_index=True)
            return new_df

        Mean_Psd_df = extract_mean_and_psd_mean(reduced_decomp_df)
        Mean_Psd_df.to_csv(f'{filename}_Mean_Psd_abg.csv')

In [15]:
%%capture
channel_name_1 = ['O1', 'O2']
channel_name_2= ['F3', 'F4','C3' ,'C4' , 'Fp1', 'Fp2']  # Add more channel names as needed

# Create dictionaries to store the filtered data
eeg_dataframe = pd.DataFrame()


# Loop through each channel and filter data
for channel_name in channel_name_2:
    eeg_channel = eeg_raw.copy().pick_channels([channel_name])
    
    # Filter for Alpha (8-13 Hz)
    alpha_filtered = eeg_channel.filter(l_freq=8, h_freq=13)
    alpha_decomp = alpha_filtered.get_data()[0]
    eeg_dataframe[f'{channel_name} alpha'] = alpha_decomp
    
    # Filter for Beta (13-30 Hz)
    beta_filtered = eeg_channel.filter(l_freq=13, h_freq=30)
    beta_decomp = beta_filtered.get_data()[0]
    eeg_dataframe[f'{channel_name} beta'] = beta_decomp
    
for channel_name in channel_name_1:
    eeg_channel = eeg_raw.copy().pick_channels([channel_name])
    
    # Filter for Gamma (30-40 Hz)
    gamma_filtered = eeg_channel.filter(l_freq=30, h_freq=40)
    gamma_decomp = gamma_filtered.get_data()[0]
    eeg_dataframe[f'{channel_name} gamma'] = gamma_decomp


segment_size = 1024
num_segments = len(eeg_dataframe) // segment_size
reduced_decomp_df = pd.DataFrame(columns=eeg_dataframe.columns)
for i in range(num_segments):
    start_idx = i * segment_size
    end_idx = (i + 1) * segment_size
    segment_data = eeg_dataframe.iloc[start_idx:end_idx]    
    mean_values = segment_data.mean()
    sampling_frequency = 1024  
    reduced_decomp_df = reduced_decomp_df.append(mean_values, ignore_index=True)

reduced_decomp_df.to_csv('alpha_beta_decomp.csv')

In [16]:
def extract_mean_and_psd_mean(reduced_decomp_df):
    new_df = pd.DataFrame(columns=['channel', 'mean', 'psd_mean'])
    req_channels = reduced_decomp_df.columns
#     df = pd.read_csv(csv_file)
    for channel in req_channels:
        if channel not in req_channels:
            continue
        mean_val = reduced_decomp_df[channel].mean()
        _, psd = welch(reduced_decomp_df[channel], fs=256)
        new_rec = {
            'channel': channel,
            'mean': mean_val,
            'psd_mean': psd.mean()
        }
        new_df = new_df.append(new_rec, ignore_index=True)
    return new_df

In [8]:
%%capture
Mean_Psd_df = extract_mean_and_psd_mean(reduced_decomp_df)

C:\Users\LENOVOi7\AppData\Local\Temp\ipykernel_15872\2839195278.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_rec, ignore_index=True)
C:\Users\LENOVOi7\AppData\Local\Temp\ipykernel_15872\2839195278.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_rec, ignore_index=True)
C:\Users\LENOVOi7\AppData\Local\Temp\ipykernel_15872\2839195278.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_rec, ignore_index=True)
C:\Users\LENOVOi7\AppData\Local\Temp\ipykernel_15872\2839195278.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(

In [9]:
Mean_Psd_df

,channel,mean,psd_mean
0,F3 alpha,4.415423e-10,8.984739e-17
1,F3 beta,1.971240e-10,1.542145e-17
2,F4 alpha,2.520625e-10,1.095181e-16
3,F4 beta,2.021261e-10,1.643429e-17
4,C3 alpha,8.446758e-11,6.022223e-17
5,C3 beta,7.577776e-11,1.033481e-17
6,C4 alpha,1.004618e-11,6.694371e-17
7,C4 beta,7.657346e-11,1.292240e-17
8,Fp1 alpha,7.977798e-12,2.482532e-20
9,Fp1 beta,2.776518e-12,5.024164e-21


In [10]:
Mean_Psd_df.to_csv('Mean_Psd_abg.csv')